# 🔑 Řešení úkolů – Lekce 3: Hash mapy a řetězce



> **Verze s kompletními řešeními** – pro lektory a kontrolu



[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

[![Python 3.8+](https://img.shields.io/badge/python-3.8+-blue.svg)](https://www.python.org/downloads/)

[![LeetCode](https://img.shields.io/badge/practice-LeetCode-orange.svg)](https://leetcode.com/)



## 📚 O tomto notebooku



Tento notebook obsahuje **kompletní řešení všech úkolů** z Lekce 3 zaměřené na hash mapy a práci s řetězci. Každé řešení navazuje na strukturu z Lekce 2 a doplňuje ji o nové postupy:



- ✅ **Funkční kód** se všemi testy

- 💡 **Komentáře a vysvětlení krok za krokem**

- 📊 **Analýza složitosti** a alternativní přístupy

- 🧪 **Automatizované testy** pomocí `unittest`

- 🎯 **Bonus řešení** pro Valid Sudoku (Medium)



### 🎓 Doporučení:

Nejdříve řeš úkoly samostatně v notebooku `lekce3.ipynb`. Teprve poté si projdi toto řešení a porovnej svůj postup.



---

## 1️⃣ Prostředí a importy



V této sekci inicializujeme prostředí a připravíme knihovny použité ve všech úlohách. Stejně jako v Lekci 2 využíváme pouze standardní knihovny Pythonu, doplněné o `collections` pro pohodlnější práci s hash mapami.

In [ ]:
from __future__ import annotations



# 📦 Standardní knihovny

from collections import Counter, defaultdict

from typing import Dict, List, Tuple

import unittest



# 📊 Vizualizace a tabulky (pro sekci 7)

import pandas as pd

import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")



# ⚙️ Helper nastavení

pd.set_option("display.max_rows", 10)

pd.set_option("display.max_colwidth", 40)

print("✅ Prostředí inicializováno – připraveno na řešení úloh.")

## 2️⃣ Načtení zadání Lekce 3



Lekce 3 nevyužívá externí datové soubory – pracujeme přímo s testovacími příklady stejnými jako v hlavním notebooku `lekce3.ipynb`. Abychom dodrželi strukturu, připravíme si datové kontejnery s ukázkovými vstupy pro všechny úkoly a ověříme, že pokrývají hraniční případy.

In [ ]:
# 📥 Ukázkové vstupy pro úkoly lekce 3

contains_duplicate_cases: List[Tuple[List[int], bool]] = [

    ([1, 2, 3, 1], True),

    ([1, 2, 3, 4], False),

    ([1, 1, 1, 3, 3, 4, 3, 2, 4, 2], True),

    ([], False),

    ([42], False),

    (list(range(10000)), False),  # velký vstup bez duplikací

]



valid_anagram_cases: List[Tuple[str, str, bool]] = [

    ("anagram", "nagaram", True),

    ("rat", "car", False),

    ("áček", "kácé", False),

    ("listen", "silent", False),  # bez normalizace velikosti písmen

    ("", "", True),

]



# Reprezentativní Sudoku desky (platná + dvě neplatné varianty)

sudoku_valid_board: List[List[str]] = [

    ["5","3",".",".","7",".",".",".","."],

    ["6",".",".","1","9","5",".",".","."],

    [".","9","8",".",".",".",".","6","."],

    ["8",".",".",".","6",".",".",".","3"],

    ["4",".",".","8",".","3",".",".","1"],

    ["7",".",".",".","2",".",".",".","6"],

    [".","6",".",".",".",".","2","8","."],

    [".",".",".","4","1","9",".",".","5"],

    [".",".",".",".","8",".",".","7","9"]

]



sudoku_row_duplicate: List[List[str]] = [

    ["8","3",".",".","7",".",".",".","."],

    ["6",".",".","1","9","5",".",".","."],

    [".","9","8",".",".",".",".","6","."],

    ["8",".",".",".","6",".",".",".","3"],  # duplicitní 8 v řádku

    ["4",".",".","8",".","3",".",".","1"],

    ["7",".",".",".","2",".",".",".","6"],

    [".","6",".",".",".",".","2","8","."],

    [".",".",".","4","1","9",".",".","5"],

    [".",".",".",".","8",".",".","7","9"]

]



sudoku_box_duplicate: List[List[str]] = [

    ["5","3",".",".","7",".",".",".","."],

    ["6",".","3","1","9","5",".",".","."],  # duplicitní 3 v boxu (0,0)

    [".","9","8",".",".",".",".","6","."],

    ["8",".",".",".","6",".",".",".","3"],

    ["4",".",".","8",".","3",".",".","1"],

    ["7",".",".",".","2",".",".",".","6"],

    [".","6",".",".",".",".","2","8","."],

    [".",".",".","4","1","9",".",".","5"],

    [".",".",".",".","8",".",".","7","9"]

]



print(f"🧾 Celkem {len(contains_duplicate_cases)} testů pro Contains Duplicate")

print(f"🧾 Celkem {len(valid_anagram_cases)} testů pro Valid Anagram")

print("✅ Sudoku testovací desky připraveny")

## 3️⃣ Řešení úkolu 3.1 – Contains Duplicate



**Obtížnost:** 🟢 Easy | **Časová složitost:** O(n) | **Prostorová složitost:** O(n)



### 💭 Myšlenkový proces

1. Nejjednodušší brute-force řešení by vyžadovalo dvojitou smyčku → O(n²).

2. Hash set umožňuje zjistit přítomnost prvku v průměru v O(1).

3. Stačí jediný průchod polem a průběžná kontrola.



### 🎯 Klíčové poznatky

- Hash set je ideální pro rychlou detekci duplicit.

- Okrajové případy: prázdné pole, jedno číslo, velký rozsah hodnot.

- Alternativně lze využít řazení, ale to zvýší časovou složitost na O(n log n).


In [ ]:
class ContainsDuplicateSolution(object):

    """Hlavní řešení – jednorázový průchod se setem."""



    def containsDuplicate(self, nums: List[int]) -> bool:

        seen = set()

        for num in nums:

            if num in seen:

                return True

            seen.add(num)

        return False





class ContainsDuplicateSortingSolution(object):

    """Alternativní řešení – řazení a porovnání sousedních prvků."""



    def containsDuplicate(self, nums: List[int]) -> bool:

        if len(nums) < 2:

            return False

        nums_sorted = sorted(nums)

        for i in range(1, len(nums_sorted)):

            if nums_sorted[i] == nums_sorted[i - 1]:

                return True

        return False





class ContainsDuplicateCounterSolution(object):

    """Varianta s Counterem – čitelná, ale se stejnou složitostí."""



    def containsDuplicate(self, nums: List[int]) -> bool:

        counts = Counter(nums)

        return any(count > 1 for count in counts.values())





main_contains_duplicate = ContainsDuplicateSolution()

alt_contains_duplicate = ContainsDuplicateSortingSolution()

counter_contains_duplicate = ContainsDuplicateCounterSolution()



print("✅ Řešení pro Contains Duplicate připravena")

## 4️⃣ Testy pro úkol 3.1



Testujeme hlavní i alternativní implementace pomocí `unittest`. Scénáře pokrývají prázdné pole, velké vstupy i typické duplicitní kombinace.

In [ ]:
class TestContainsDuplicate(unittest.TestCase):

    def setUp(self) -> None:

        self.solutions = [

            ("hash_set", main_contains_duplicate.containsDuplicate),

            ("sorting", alt_contains_duplicate.containsDuplicate),

            ("counter", counter_contains_duplicate.containsDuplicate),

        ]



    def test_contains_duplicate_cases(self):

        for label, fn in self.solutions:

            with self.subTest(solution=label):

                for nums, expected in contains_duplicate_cases:

                    self.assertEqual(

                        fn(nums[:]),

                        expected,

                        msg=f"Solution '{label}' selhala pro vstup {nums[:10]}...",

                    )





suite_cd = unittest.TestLoader().loadTestsFromTestCase(TestContainsDuplicate)

result_cd = unittest.TextTestRunner(verbosity=1).run(suite_cd)

assert result_cd.wasSuccessful(), "Testy pro Contains Duplicate neprošly!"

## 5️⃣ Řešení úkolu 3.2 – Valid Anagram



**Obtížnost:** 🟢 Easy | **Časová složitost:** O(n) | **Prostorová složitost:** O(1) až O(k)



### 💭 Myšlenkový proces

1. Nejprve ověříme délky – rozdílné délky znamenají okamžité `False`.

2. Poté sčítáme/odčítáme četnost znaků pomocí slovníku nebo `Counter`.

3. Alternativně lze řetězce seřadit a porovnat – přehledné, ale pomalejší.



### 🎯 Klíčové poznatky

- `collections.Counter` nabízí nejkratší a stále efektivní řešení.

- Pokud ignorujeme velikost písmen nebo diakritiku, je nutná normalizace.

- V multi-jazyčných vstupních datech je důležité pracovat s unicode znaky správně.

In [ ]:
class ValidAnagramSolution(object):

    """Hlavní řešení – manuální počítání znaků pomocí slovníku."""



    def isAnagram(self, s: str, t: str) -> bool:

        if len(s) != len(t):

            return False

        counts: Dict[str, int] = defaultdict(int)

        for char_s, char_t in zip(s, t):

            counts[char_s] += 1

            counts[char_t] -= 1

        return all(value == 0 for value in counts.values())





class ValidAnagramCounterSolution(object):

    """Alternativní řešení – dvě Counter struktury."""



    def isAnagram(self, s: str, t: str) -> bool:

        return Counter(s) == Counter(t)





class ValidAnagramSortedSolution(object):

    """Řešení pomocí seřazení – nejjednodušší na pochopení."""



    def isAnagram(self, s: str, t: str) -> bool:

        return sorted(s) == sorted(t)





main_valid_anagram = ValidAnagramSolution()

counter_valid_anagram = ValidAnagramCounterSolution()

sorted_valid_anagram = ValidAnagramSortedSolution()



print("✅ Řešení pro Valid Anagram připravena")

## 6️⃣ Testy pro úkol 3.2



Testujeme všechny tři varianty řešení a přidáváme scénář s normalizací velikosti písmen, abychom ukázali, jak lze algoritmus rozšířit.

In [ ]:
class TestValidAnagram(unittest.TestCase):

    def setUp(self) -> None:

        self.solutions = [

            ("dict_diff", main_valid_anagram.isAnagram),

            ("counter", counter_valid_anagram.isAnagram),

            ("sorted", sorted_valid_anagram.isAnagram),

        ]



    def test_valid_anagram_cases(self):

        for label, fn in self.solutions:

            with self.subTest(solution=label):

                for s, t, expected in valid_anagram_cases:

                    self.assertEqual(

                        fn(s, t),

                        expected,

                        msg=f"Solution '{label}' selhala pro dvojici ({s}, {t})",

                    )



    def test_case_insensitive_extension(self):

        """Ukázka, jak snadno doplnit case-insensitive variantu."""



        def case_insensitive_check(s: str, t: str) -> bool:

            return main_valid_anagram.isAnagram(s.lower(), t.lower())



        self.assertTrue(case_insensitive_check("Listen", "Silent"))

        self.assertFalse(case_insensitive_check("Python", "Java"))





suite_anagram = unittest.TestLoader().loadTestsFromTestCase(TestValidAnagram)

result_anagram = unittest.TextTestRunner(verbosity=1).run(suite_anagram)

assert result_anagram.wasSuccessful(), "Testy pro Valid Anagram neprošly!"

## 7️⃣ Vizualizace výsledků úkolu 3.2



Na závěr shrneme testovací případy pro anagramy do tabulky a vykreslíme jednoduchý graf, který ukáže, pro které dvojice řetězců vychází výsledek `True`.

In [1]:
anagram_df = pd.DataFrame(

    [

        {"s": s, "t": t, "expect": expected}

        for s, t, expected in valid_anagram_cases

    ]

)

anagram_df["pair"] = anagram_df["s"] + " ↔ " + anagram_df["t"]

display(anagram_df[["pair", "expect"]])



fig, ax = plt.subplots(figsize=(6, 3))

ax.bar(anagram_df["pair"], anagram_df["expect"].astype(int), color="#4f86f7")

ax.set_ylabel("Je anagram (1 = ano)")

ax.set_xticklabels(anagram_df["pair"], rotation=45, ha="right")

ax.set_title("Výsledky testů pro Valid Anagram")

plt.tight_layout()



# Uložení grafu do složky lekce 3

output_path = r"c:\Users\Uzivatel\SELF LEARN\tynaj\tynaj\Lekce3\valid_anagram_summary.png"

fig.savefig(output_path, dpi=150)

print(f"📈 Graf uložen do souboru: {output_path}")

NameError: name 'pd' is not defined

---



## 🧠 BONUS řešení – Valid Sudoku



**Obtížnost:** 🟡 Medium | **Časová složitost:** O(1) | **Prostorová složitost:** O(1)



### 💭 Myšlenkový proces

1. Každé číslo musí být unikátní v řádku, sloupci i 3×3 boxu.

2. Využijeme tři slovníky mapující index → set viděných hodnot.

3. Při nalezení duplicity okamžitě vracíme `False`.



### 🎯 Klíčové poznatky

- Pro box stačí index `(row // 3, col // 3)`.

- Sudoku má fixní velikost, takže časová složitost je konstantní.

- Varianty: bitmasking pro úsporu paměti nebo pre-allokované seznamy délky 9.

In [ ]:
class ValidSudokuSolution(object):

    """Hlavní řešení – trojice slovníků se sety."""



    def isValidSudoku(self, board: List[List[str]]) -> bool:

        rows = [set() for _ in range(9)]

        cols = [set() for _ in range(9)]

        boxes = defaultdict(set)



        for r in range(9):

            for c in range(9):

                value = board[r][c]

                if value == ".":

                    continue

                box_key = (r // 3, c // 3)

                if value in rows[r] or value in cols[c] or value in boxes[box_key]:

                    return False

                rows[r].add(value)

                cols[c].add(value)

                boxes[box_key].add(value)

        return True





class ValidSudokuBitmaskSolution(object):

    """Alternativní řešení – reprezentace pomocí bitmask."""



    def isValidSudoku(self, board: List[List[str]]) -> bool:

        rows = [0] * 9

        cols = [0] * 9

        boxes = [0] * 9



        for r in range(9):

            for c in range(9):

                value = board[r][c]

                if value == ".":

                    continue

                mask = 1 << (ord(value) - ord("1"))

                box_idx = (r // 3) * 3 + (c // 3)

                if rows[r] & mask or cols[c] & mask or boxes[box_idx] & mask:

                    return False

                rows[r] |= mask

                cols[c] |= mask

                boxes[box_idx] |= mask

        return True





main_valid_sudoku = ValidSudokuSolution()

bitmask_valid_sudoku = ValidSudokuBitmaskSolution()



print("✅ Řešení pro Valid Sudoku připravena")

### 🧪 Testy pro BONUS úlohu

Testujeme standardní i bitmaskové řešení na připravených deskách.

In [ ]:
class TestValidSudoku(unittest.TestCase):

    def setUp(self) -> None:

        self.solutions = [

            ("set_based", main_valid_sudoku.isValidSudoku),

            ("bitmask", bitmask_valid_sudoku.isValidSudoku),

        ]



    def test_valid_board(self):

        for label, fn in self.solutions:

            with self.subTest(solution=label):

                self.assertTrue(fn(sudoku_valid_board))



    def test_row_duplicate(self):

        for label, fn in self.solutions:

            with self.subTest(solution=label):

                self.assertFalse(fn(sudoku_row_duplicate))



    def test_box_duplicate(self):

        for label, fn in self.solutions:

            with self.subTest(solution=label):

                self.assertFalse(fn(sudoku_box_duplicate))





suite_sudoku = unittest.TestLoader().loadTestsFromTestCase(TestValidSudoku)

result_sudoku = unittest.TextTestRunner(verbosity=1).run(suite_sudoku)

assert result_sudoku.wasSuccessful(), "Testy pro Valid Sudoku neprošly!"

---



## 📊 Shrnutí a porovnání řešení



| Úloha | Obtížnost | Hlavní technika | Časová složitost | Prostorová složitost |

|-------|-----------|-----------------|------------------|----------------------|

| Contains Duplicate | 🟢 Easy | Hash set | O(n) | O(n) |

| Valid Anagram | 🟢 Easy | Hash map / Counter | O(n) | O(1) – O(k) |

| Valid Sudoku | 🟡 Medium | Hash map + indexování | O(1) | O(1) |



### 🔑 Klíčové takeaways

1. **Hash struktury** zajišťují konstantní čas pro lookup.

2. **Counter** je rychlý způsob, jak porovnat četnosti znaků.

3. **Indexování 3×3 boxů** v Sudoku lze snadno generovat pomocí celočíselného dělení.

4. **Automatizované testy** pomáhají ověřit více variant řešení najednou.



### 🚀 Doporučení pro další trénink

- **Group Anagrams**, **Top K Frequent Elements**, **Longest Substring Without Repeating Characters**

- Vyzkoušej implementovat Sudoku validaci pomocí čisté bitové aritmetiky bez pomocných seznamů.



---



*Skvěle zvládnutá práce s hash mapami! Pokračuj v pravidelném procvičování, ať jsou tyto techniky naprosto automatické.* 💪